In [1]:
import os
import numpy as np
from PIL import Image

# Define paths to the pseudo label folders
folder1 = r'C:\Users\arunm\thusharproj\test\pseudomodel1'
folder2 = r'C:\Users\arunm\thusharproj\test\pseudomodel2'

# Get the list of pseudo label files
files_model1 = sorted(os.listdir(folder1))
files_model2 = sorted(os.listdir(folder2))

# Ensure both folders have the same number of files
assert len(files_model1) == len(files_model2), "Mismatch in the number of pseudo label files."

# Load the pseudo labels
def load_image_as_array(file_path):
    """Load an image file as a NumPy array."""
    with Image.open(file_path) as img:
        return np.array(img)

pseudo_labels_model1 = [load_image_as_array(os.path.join(folder1, f)) for f in files_model1]
pseudo_labels_model2 = [load_image_as_array(os.path.join(folder2, f)) for f in files_model2]

# Verify the loading
print(f"Loaded {len(pseudo_labels_model1)} pseudo labels from model 1.")
print(f"Loaded {len(pseudo_labels_model2)} pseudo labels from model 2.")

# Calculate agreement (assuming class labels)
agreement_mask = [pseudo_labels_model1[i] == pseudo_labels_model2[i] for i in range(len(pseudo_labels_model1))]

final_pseudo_labels = []

for i in range(len(agreement_mask)):
    # Keep labels where both models agree, otherwise set to 0 (or some background label)
    final_label = np.where(agreement_mask[i], pseudo_labels_model1[i], 0)
    final_pseudo_labels.append(final_label)

# Define the output folder and create it if it doesn't exist
output_folder = r'C:\Users\arunm\thusharproj\test\final_pseudo_labels'
os.makedirs(output_folder, exist_ok=True)

# Ensure the length of final_pseudo_labels matches files_model1 before saving
if len(final_pseudo_labels) != len(files_model1):
    raise ValueError(f"Mismatch between number of labels ({len(final_pseudo_labels)}) and number of files ({len(files_model1)})")

# Save the final pseudo labels as PNG images
for i, pseudo_label in enumerate(final_pseudo_labels):
    # Convert the NumPy array to an image
    pseudo_label_img = Image.fromarray(pseudo_label.astype(np.uint8))
    # Save the image as a PNG file
    pseudo_label_img.save(os.path.join(output_folder, f"{os.path.splitext(files_model1[i])[0]}.png"))

print(f"Saved {len(final_pseudo_labels)} pseudo labels as PNG files in {output_folder}.")


Loaded 505 pseudo labels from model 1.
Loaded 505 pseudo labels from model 2.
Saved 505 pseudo labels as PNG files in C:\Users\arunm\thusharproj\test\final_pseudo_labels.


In [2]:
#combining labled and psedolabled data
import os
import shutil

def combine_folders(source_folders, destination_folder):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Track the index for the filenames to maintain order
    index = 0
    
    for folder in source_folders:
        # List all files in the current folder
        files = sorted(os.listdir(folder))
        
        for file_name in files:
            # Construct full file path
            source_file = os.path.join(folder, file_name)
            # Generate new file name with index to maintain order
            dest_file = os.path.join(destination_folder, f"{index:06d}_{file_name}")
            # Copy file to the destination folder
            shutil.copy2(source_file, dest_file)
            index += 1

# Example usage:
source_folders1 = [
    r"C:\Users\arunm\thusharproj\test\2018LA_Seg_Training Set\images\10",
    r"C:\Users\arunm\thusharproj\test\2018LA_Seg_Training Set\images\train3"
]
destination_folder1 = r"C:\Users\arunm\thusharproj\test\finaldataset\images"

combine_folders(source_folders1, destination_folder1)

source_folders2 = [
    r"C:\Users\arunm\thusharproj\test\2018LA_Seg_Training Set\annotations\10",
    r"C:\Users\arunm\thusharproj\test\final_pseudo_labels"
]
destination_folder2 = r"C:\Users\arunm\thusharproj\test\finaldataset\lables"

combine_folders(source_folders2, destination_folder2)